[Data Visualization](https://infovis.fh-potsdam.de/tutorials/) · FH Potsdam · Summer 2023

# Tutorial 7: Many dimensions

In this tutorial we are going to analyze multidimensional datasets. In the context of computing, ‘multidimensional’ refers to data that contains multiple attributes per item. When working with such multidimensional data we face a range of challenges from data preparation to processing and presentation. Let's look at these steps one after another.

## 🛒 1. Prepare 




In [1]:
import pandas as pd
import altair as alt

First, data preparation can be more elaborate when we are trying to make sense of many dimensions. We may encounter multidimensional data by 
- importing a dataset that contains multiple dimensions (easiest case),
- working with data from multiple sources (can be challenging), or
- querying multiple data fields from one database (may be difficult due to the query language). 

We will take a look at a range of national statistics about different sets of countries and thus go through each of these cases.

### OECD data

Of course, it would be most convenient if we had a dataset that already contains all the dimensions that we are interested in and that can be loaded directly into a Pandas DataFrame. Sometimes this is the case. For example, the [OECD Better Life Index](http://www.oecdbetterlifeindex.org/) about the prosperity, health, and development of its member states is available as one file. It comprises development data about 36 countries and 24 dimensions spanning social, economic, and ecological aspects of well-being.

The dataset exists online as a CSV file, so we can simply load it via `read_csv(url)`:

In [2]:
url = "https://gist.github.com/scotthmurray/f71065a5694f22259bf9/raw/ce891b9fe7ec3c5cab3308f4cd0c8eeccc36f6c7/Better%2520Life%2520Index%2520Data.csv"
oecd = pd.read_csv(url)

# to keep things consistent with the other examples we turn column names to lowercase
oecd.columns = map(str.lower, oecd.columns)

✏️ *Examine the dataset, e.g., by using `head()`, `info()` and `describe()`:*

### Multiple sources

Having everything in one file is the easiest and maybe rarest case of data preparation. More often we have to work with multiple datasets each containing different aspects about entities of interest. Suppose, we would like to work with various country statistics, ranging from population numbers and area sizes to economic performance and even Covid–19 cases. 

To do this, we will load data from three sources. In order to integrate the data sources, we need to match information belonging to the same country. For this we will use three-letter country codes ([ISO 3166-1 alpha-3](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3)). The country codes allow us to reliably cross-reference the datasets, without confusing different ways of spelling or naming countries (such as USA, United States, US, etc). However, note that this does not resolve territorial conflicts that, of course, do exist. 

Let's start with the basics: we will use the handy `countryInfoCSV` service by GeoNames to get population numbers and area sizes:

In [3]:
geonames_full = pd.read_csv("https://www.geonames.org/countryInfoCSV", sep='\t', keep_default_na=False)
geonames_full.head()

,iso alpha2,iso alpha3,iso numeric,fips code,name,capital,areaInSqKm,population,continent,languages,currency,geonameId
0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU,ca,EUR,3041565
1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,9630959,AS,"ar-AE,fa,en,hi,ur",AED,290557
2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,37172386,AS,"fa-AF,ps,uz-AF,tk",AFN,1149361
3,AG,ATG,28,AC,Antigua and Barbuda,St John's,443.0,96286,NA,en-AG,XCD,3576396
4,AI,AIA,660,AV,Anguilla,The Valley,102.0,13254,NA,en-AI,XCD,3573511


Look! Did you notice that the GeoNames dataset contains a column called `iso alpha 3`? This is our friend, who will link to the other datasets! So we will keep it, together with three other columns, which might come in handy.

In [4]:
geonames = geonames_full[['name', 'iso alpha3', 'areaInSqKm', 'population']]

And for simplicity sake we rename them, after which we set the country `code` as the DataFrame's index:

In [5]:
geonames.columns = ["country", "code", "area", "population"]
geonames = geonames.set_index("code")
geonames

,country,area,population
code,,,
AND,Andorra,468.0,77006
ARE,United Arab Emirates,82880.0,9630959
AFG,Afghanistan,647500.0,37172386
ATG,Antigua and Barbuda,443.0,96286
AIA,Anguilla,102.0,13254
...,...,...,...
YEM,Yemen,527970.0,28498687
MYT,Mayotte,374.0,279471
ZAF,South Africa,1219912.0,57779622


Next we download GDP (gross domestic product) statistics from the World Bank, which provides such datasets [via their open data portal](https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?view=map). Here we simply load it with `read_csv()` from a local file already downloaded for your convencience:

In [6]:
worldbank_full = pd.read_csv("worldbank_gdp.csv", header=2)


The dataset contains GDP statistics over several years; in this tutorial we focus on 2020. Again, we select and rename the columns and set the country `code` as the DataFrame's index:

In [7]:
worldbank = worldbank_full[ ["Country Code", "2020"] ]
worldbank.columns = ["code", "gdp"]
worldbank = worldbank.set_index("code")
worldbank

,gdp
code,
ABW,NaN
AFE,9.207923e+11
AFG,2.011614e+10
AFW,7.845876e+11
AGO,5.837598e+10
...,...
XKX,7.716925e+09
YEM,NaN
ZAF,3.354421e+11


Last but not least, we would like to include Covid–19 statistics, which are collected for many countries by the European Centre for Disease Prevention and Control (ECDC):

In [8]:
# https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide
ecdc_full = pd.read_csv("https://opendata.ecdc.europa.eu/covid19/casedistribution/csv")

By now you know, what we'll do: select and rename the columns, and turn the country code into the index:

In [9]:
ecdc = ecdc_full[ ["cases", "deaths", "countryterritoryCode"] ]
ecdc.columns = ["covid19_cases", "covid19_deaths", "code"]
ecdc = ecdc.set_index("code")

The Covid–19 statistics are provided on a daily basis. For the purpose of this tutorial, we are only interested in total numbers:

In [10]:
ecdc = ecdc.groupby("code").sum()
ecdc

,covid19_cases,covid19_deaths
code,,
ABW,5049,46
AFG,49273,1971
AGO,16188,371
AIA,10,0
ALB,48530,1003
...,...,...
XKX,46580,1198
YEM,2083,606
ZAF,860964,23276


To integrate the three DataFrames we need to make sure that the values associated with a specific country are linked to the right country. 

We already set the country codes as the indices for the three DataFrames. By virtue of using this identifier we can now simply integrate the various data sources using the `join()` method, which by default uses the indices of the DataFrames.


In [11]:
multiple = geonames.join(worldbank).join(ecdc)

# from now on, we do not need the country codes anymore; we can remove them:
multiple = multiple.reset_index(drop=True)
multiple

,country,area,population,gdp,covid19_cases,covid19_deaths
0,Andorra,468.0,77006,NaN,7338.0,79.0
1,United Arab Emirates,82880.0,9630959,3.588688e+11,184949.0,617.0
2,Afghanistan,647500.0,37172386,2.011614e+10,49273.0,1971.0
3,Antigua and Barbuda,443.0,96286,1.370281e+09,148.0,4.0
4,Anguilla,102.0,13254,NaN,10.0,0.0
...,...,...,...,...,...,...
247,Yemen,527970.0,28498687,NaN,2083.0,606.0
248,Mayotte,374.0,279471,NaN,NaN,NaN
249,South Africa,1219912.0,57779622,3.354421e+11,860964.0,23276.0
250,Zambia,752614.0,17351822,1.811063e+10,18274.0,367.0


Some of the later steps require values to be present for each country; with `dropna()` we can remove all rows with missing values.


✏️ *But before we remove incomplete rows, let's do a quick check and take a look at those rows with missing values in any column (see some hints [over here](https://datatofish.com/rows-with-nan-pandas-dataframe/)). Please, note that this does not work if you already ran the following code block, which removes rows with missing values:*

Removing incomplete rows keeps the old index, which we can reset yet again.

In [13]:
multiple = multiple.dropna().reset_index(drop=True)
multiple

,country,area,population,gdp,covid19_cases,covid19_deaths
0,United Arab Emirates,82880.0,9630959,3.588688e+11,184949.0,617.0
1,Afghanistan,647500.0,37172386,2.011614e+10,49273.0,1971.0
2,Antigua and Barbuda,443.0,96286,1.370281e+09,148.0,4.0
3,Albania,28748.0,2866376,1.488763e+10,48530.0,1003.0
4,Armenia,29800.0,2951776,1.264121e+10,148682.0,2503.0
...,...,...,...,...,...,...
179,Vanuatu,12200.0,292680,8.815479e+08,1.0,0.0
180,Kosovo,10908.0,1845300,7.716925e+09,46580.0,1198.0
181,South Africa,1219912.0,57779622,3.354421e+11,860964.0,23276.0
182,Zambia,752614.0,17351822,1.811063e+10,18274.0,367.0


### Wikidata query

Apart from working with specific datasets, we can also generate custom data sources by formulating queries against databases.

For various topics, there are comprehensive databases that offer multidimensional information across a wide range of domains. Wikipedia's sister Wikidata is a particularly broad knowledge base that can be queried using the query language SPARQL. The query syntax may seem a bit daunting, which is because it operates on the basis of triples representing semantic relationships binding entities. In this tutorial, we will not go deeper into the syntax, but it may already help to visit the web-based interface to the [Wikidata query service](https://query.wikidata.org).

Queries against Wikidata can be directly issued from the comfort of your notebook using HTTP requests! For this we need the `requests` library again, the URL of the endpoint and the SPARQL query. The following query requests three bits of information about European countries: `gdp`, `area` and `population`.

In [14]:
import requests
import requests_cache # this package creates a local cache, which speeds things up quite a bit

# set up cache for web requests
requests_cache.install_cache('cache')

endpoint = "https://query.wikidata.org/sparql"

# triple quotes start and end multi-line strings
sparql = """
SELECT ?countryLabel ?population ?gdp ( MAX(?areas) AS ?area )
WHERE {
  ?country wdt:P31 wd:Q3624078;
           wdt:P463 wd:Q458;
           wdt:P1082 ?population;
           wdt:P8744/wdt:P2131 ?gdp;
           p:P2046/psn:P2046/wikibase:quantityAmount ?areas.
  
  ?gdp_statement ps:P2131 ?gdp;
                pq:P585 ?gdp_date. 

  FILTER NOT EXISTS {
    ?country p:P2131/pq:P585 ?gdp_date_ .
    FILTER (?gdp_date_ > ?gdp_date)
  }
  
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
  }
}

GROUP BY ?countryLabel ?area ?population ?gdp

"""

res = requests.get(endpoint, params = {'format': 'json', 'query': sparql})
response = res.text

In [15]:
print(response[:1000])

{
  "head" : {
    "vars" : [ "countryLabel", "population", "gdp", "area" ]
  },
  "results" : {
    "bindings" : [ {
      "countryLabel" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Finland"
      },
      "area" : {
        "datatype" : "http://www.w3.org/2001/XMLSchema#decimal",
        "type" : "literal",
        "value" : "390905420000"
      },
      "population" : {
        "datatype" : "http://www.w3.org/2001/XMLSchema#decimal",
        "type" : "literal",
        "value" : "5516224"
      },
      "gdp" : {
        "datatype" : "http://www.w3.org/2001/XMLSchema#decimal",
        "type" : "literal",
        "value" : "251884887972.766"
      }
    }, {
      "countryLabel" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Croatia"
      },
      "area" : {
        "datatype" : "http://www.w3.org/2001/XMLSchema#decimal",
        "type" : "literal",
        "value" : "56594000000"
      },
      "population" : {
      

The query might take a bit of time to finish (a minute or so), which is due to the various attributes that are queried. Once the request is finished, we can parse the results:

In [16]:
import json

# let's define the parsing steps as a function, so that you can reuse it later
def wikidata_to_dataframe(data):

  # parse json data from response and get results:
  results = json.loads(data)["results"]["bindings"]

  # column names we draw from the first result
  cols = [ val for val in results[0] ]

  rows = []
  
  # to get the values from this, we need to loop through the results:
  for result in results:
      values = [ result[val]["value"] for val in result ]
      rows.append(values)

  # with rows and cols we can create a DataFrame:
  return pd.DataFrame(rows, columns=cols)

wikidata = wikidata_to_dataframe(response)
wikidata = wikidata.rename(columns={'countryLabel':'country'})
wikidata

,country,area,population,gdp
0,Finland,390905420000,5516224,251884887972.766
1,Croatia,56594000000,3871833,54849180228.8716
2,Romania,238397000000,19053815,211803281924.738
3,Sweden,528861060000,10409248,538040458216.997
4,Malta,316000000,465292,12537750732.4769
5,Republic of Ireland,69797000000,5123536,333730764773.18
6,Latvia,64593790000,1875757,30264454641.8004
7,Estonia,45339000000,1365884,30312000000
8,Austria,83878990000,8979894,447033100107
9,Germany,357587770000,83149300,3677439129776.6


At this point the columns are not treated as series of numeric values yet, but strings (displayed as object). See:

In [17]:
wikidata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   country     23 non-null     object
 1   area        23 non-null     object
 2   population  23 non-null     object
 3   gdp         23 non-null     object
dtypes: object(4)
memory usage: 864.0+ bytes


To let the DataFrame know that three of these columns contain numbers, we apply the `to_numeric` method to all columns except the first one, which contains the country names:

In [18]:
cols = wikidata.columns[1:]
wikidata[cols] = wikidata[cols].apply(pd.to_numeric)
wikidata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     23 non-null     object 
 1   area        23 non-null     int64  
 2   population  23 non-null     int64  
 3   gdp         23 non-null     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 864.0+ bytes


✏️ *What would you query from Wikidata? Have a look at the examples of the [Wikidata Query Service](https://query.wikidata.org) for inspiration and use the following template to run the query in this notebook. Wikidata now also has a [query builder](https://query.wikidata.org/querybuilder/), which might help to formulate Sparql queries.*

In [19]:
sparql = """

# 1. enter your query here

"""

# 2. uncomment the following lines

# res = requests.get(endpoint, params = {'format': 'json', 'query': sparql})
# your_wikidata = wikidata_to_dataframe(res.text)
# your_wikidata

In case you're curious about the text colors in above code cell: the string content of the SPARQL query can span multiple lines, therefore it is rendered in red. The lines below are displayed in green or blue (depending where you read the tutorials), because they are *commented out* via the hash sign: **#**. You can quickly comment out (disable) by adding a hash and uncomment (enable) it by removing it. To do the pencil exercise above, you need to remove the hashsigns in the last three lines.

## 🗂 2. Process

There are several processing steps we can apply to the multidimensional data, to visualize them. First, we can take a  look at the various correlations between dimensions, then we will try out two techniques for dimensionality reduction.

### Correlation analysis

Regardless of the source, once we have multidimensional data we can explore how the quantitative  dimensions relate to one another, i.e., how they correlate. The `corr()` method of Pandas faciliates this process by calculating the coefficients of pairwise correlations between data columns…

In [20]:
oecd.corr()

,dwellings without basic facilities,housing expenditure,rooms per person,household net adjusted disposable income,household net financial wealth,employment rate,job security,long-term unemployment rate,personal earnings,quality of support network,...,water quality,consultation on rule-making,voter turnout,life expectancy,self-reported health,life satisfaction,assault rate,homicide rate,employees working very long hours,time devoted to leisure and personal care
dwellings without basic facilities,1.000000,-0.489839,-0.567042,-0.593321,-0.388205,-0.287332,-0.123392,-0.166020,-0.629090,-0.424416,...,-0.738332,-0.480795,0.049858,-0.743453,-0.581215,-0.481953,0.271740,0.459555,0.413667,-0.376653
housing expenditure,-0.489839,1.000000,0.108116,0.038185,0.075931,-0.087133,0.173189,0.266434,0.031752,0.064197,...,0.283561,0.227886,-0.143251,0.321563,0.391433,0.090239,-0.119760,-0.252707,-0.045215,0.023785
rooms per person,-0.567042,0.108116,1.000000,0.759664,0.544616,0.474966,-0.068846,-0.158453,0.785113,0.610136,...,0.668145,0.317098,0.268994,0.595731,0.556736,0.602806,-0.365941,-0.341003,-0.285292,0.355559
household net adjusted disposable income,-0.593321,0.038185,0.759664,1.000000,0.731852,0.488977,-0.170100,-0.219721,0.924154,0.501033,...,0.610765,0.238383,0.208298,0.622414,0.486712,0.555455,-0.417265,-0.449719,-0.332307,0.266777
household net financial wealth,-0.388205,0.075931,0.544616,0.731852,1.000000,0.393753,-0.213427,-0.231309,0.699240,0.340065,...,0.430827,0.078099,-0.013059,0.515270,0.320130,0.412750,-0.205766,-0.291377,-0.092281,0.089523
employment rate,-0.287332,-0.087133,0.474966,0.488977,0.393753,1.000000,-0.581980,-0.634556,0.485224,0.628297,...,0.623944,0.140697,0.063188,0.329780,0.209322,0.747185,-0.233353,-0.085511,-0.279120,0.195197
job security,-0.123392,0.173189,-0.068846,-0.170100,-0.213427,-0.581980,1.000000,0.718938,-0.179853,-0.216039,...,-0.265321,0.048292,-0.141330,0.059452,0.191610,-0.405422,0.011376,-0.112623,-0.050162,0.153720
long-term unemployment rate,-0.166020,0.266434,-0.158453,-0.219721,-0.231309,-0.634556,0.718938,1.000000,-0.165461,-0.269344,...,-0.215873,-0.100951,-0.296382,0.025290,-0.006083,-0.588429,-0.051477,-0.204423,-0.273848,0.298353
personal earnings,-0.629090,0.031752,0.785113,0.924154,0.699240,0.485224,-0.179853,-0.165461,1.000000,0.534175,...,0.652255,0.322097,0.228281,0.681086,0.492508,0.568969,-0.443623,-0.522366,-0.323937,0.343777
quality of support network,-0.424416,0.064197,0.610136,0.501033,0.340065,0.628297,-0.216039,-0.269344,0.534175,1.000000,...,0.668132,0.146287,0.137378,0.408093,0.380586,0.588929,-0.385733,-0.343444,-0.445918,0.371897


A positive value just under 1 indicates a strong positive correlation, and a negative value just above -1 indicates an inverse correlation. For example, for the OECD data, *rooms per person* is relatively strongly correlated with *personal earnings*. Can you spot another strong negative or positive correlation? Later we will find a way to visualize such a correlation matrix.

✏️ *Run correlation analyses on any of the other DataFrames we prepared above:*

You might encounter high correlations between covid–19 numbers and gdp - remember & repeat after me: *correlation does not mean causation!*

### PCA

Next, we are going to look at two dimensionality reduction techniques that *project* a high-dimensional dataset onto a plane.

[Principal component analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) (PCA) is a data processing technique that can be used to reduce a high-dimensional dataset to a few — typically two — main components. While a given dataset may contain many different dimensions, PCA is used to extract those main axes that help to differantiate the data points the best. Let's do this with the OECD data!


In [21]:
# import a few components from the machine learning library scikit-learn:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# create a data pipeline, which includes scaling to normalize the data
# and by initializing the PCA with the number of principal components: 2
pipe = Pipeline([('scaling', StandardScaler()), ('pca', PCA(n_components=2))])

# run the PCA on all columns (except the first which is the country name)
principal_components = pipe.fit_transform(oecd[oecd.columns[1:]])

# let's have a look
principal_components

array([[-3.1412375 , -1.16806912],
       [-1.81372201, -1.46980565],
       [-1.81756929, -0.47223677],
       [ 4.88261036, -2.84192496],
       [-3.39055718, -0.36589308],
       [ 4.19239801, -1.96566591],
       [ 0.39268858,  0.89939301],
       [-2.92025017, -0.07138337],
       [ 2.02998178,  0.63200076],
       [-2.4734093 ,  0.4890045 ],
       [-0.95910409,  0.1214977 ],
       [-2.27854523, -0.54517468],
       [ 3.10165962,  5.46032054],
       [ 2.56219186,  1.98623689],
       [-2.09358533, -1.23164587],
       [-1.76896703,  1.82271137],
       [ 1.50777355, -1.29780572],
       [ 0.27356567,  1.15774084],
       [-0.65385875, -0.44433129],
       [ 1.39581363, -0.55863067],
       [-1.72106825, -1.35882664],
       [ 6.42749295, -3.28152975],
       [-2.57220852, -0.80199977],
       [-2.24846402,  0.01581683],
       [-2.82842126, -1.50354461],
       [ 1.72656611,  2.00831992],
       [ 1.83323291,  2.21039445],
       [ 5.41805788, -2.01338784],
       [ 1.61020243,

The PCA returns the positions in the same order as the input dataset, which means we can simply combine these coordinates with the rest of the data to visualize it in a scatterplot:

In [22]:
# first turn principal_components into a DataFrame
pca_positions = pd.DataFrame(principal_components, columns = ['x', 'y'])

# … and combine it with the source dataframe oecd
oecd_pca = pd.concat([oecd, pca_positions], axis = 1)

Last but not least, we generate a scatterplot, using the x and y values from the PCA:


In [23]:
alt.Chart(oecd_pca).mark_circle().encode(
    x=alt.X('x', axis=alt.Axis(labels=False)),
    y=alt.Y('y', axis=alt.Axis(labels=False)),    
    tooltip='country'
).properties(width=400, height=400)

alt.Chart(...)

✏️ *Run the PCA analysis on any of the other multidimensional datasets we created above:*

In [24]:
# 1. run PCA on the numeric columns


# 2. combine the positions with original data


# 3. display a scatterplot



### UMAP

One major downside of PCA is that it throws away a lot of dimensionality information not included in the few (often two) principal components that are selected. A more recently developed dimensionality reduction technique is [UMAP](https://umap-learn.readthedocs.io/) (Uniform manifold approximation and projection). With this projection technique all dimensions are considered — as the name suggests: *in approximation* — to generate a dimensionality reduction that satisfies both global and local structures. We keep it brief here, but [there is more to it](https://pair-code.github.io/understanding-umap/).

In [25]:
# import the umap library
import umap.umap_ as umap

# there are a bunch of warnings generated by the umap package that we will omit
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")

# there are two main parameters, which you need to tweak
reducer = umap.UMAP(
        n_neighbors=15,  # balances local versus global structure
        min_dist=.4  # .01 for tight clumps, large for loose 1
)

# the datasets have a similar structure; you can replace it with multiple or wikidata:
data = oecd

# we use again the StandardScaler for normalization
# and add the UMAP reducer afterwards into the pipeline
pipe = Pipeline([('scaling', StandardScaler()), ('umap', reducer)])

# start the normalization and reduction steps
embedding = pipe.fit_transform(data[data.columns[1:]])

# turn the resulting embedding into dataframe, in which the positions are x and y
umap_positions = pd.DataFrame(embedding, columns=["x", "y"])

# … and merge it with the original dataset
data_umap = pd.concat([data, umap_positions], axis = 1)

# to center the scatterplot around generated positions,
# we adjust the scales according to the smallest and largest x and y values:
x_domain = [data_umap["x"].min(), data_umap["x"].max() ]
y_domain = [data_umap["y"].min(), data_umap["y"].max() ]

/root/venv/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/root/venv/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/root/venv/lib/python3.9/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argument 

In [26]:
# display scatterplot and pass domains for x and y to scale parameter
alt.Chart(data_umap).mark_circle().encode(
    alt.X('x', scale=alt.Scale(domain=x_domain), axis=alt.Axis(labels=False)),
    alt.Y('y', scale=alt.Scale(domain=y_domain), axis=alt.Axis(labels=False)),
    tooltip="country"
).properties(width=400, height=400)

alt.Chart(...)

✏️ *Play with the two main parameters and observe the various arrangements for the `oecd`, `multiple`, and `wikidata` datasets!*

## 🥗 3. Present

With the above dimensionality reduction techniques (short projections) we have already started to visualize the data. So we skip the simple scatterplot here…

 ### Scatterplot matrix

… but we can have multiple scatterplots in a matrix layout, each cell containing a small scatterplot for a pair of dimensions.

Altair supports [repeated charts](https://altair-viz.github.io/user_guide/compound_charts.html?#repeat-chart), with which we can create multi-view displays in a snap:

In [27]:
data = wikidata

cols = ["area",	"population", "gdp"]


alt.Chart(data).mark_circle().encode(
    # the data dimensions used for the encoding are specified below under repeat
    # they are all quantitative and we remove the axis labels to avoid clutter
    alt.X(alt.repeat("column"), type='quantitative', axis=alt.Axis(labels=False)),
    alt.Y(alt.repeat("row"), type='quantitative', axis=alt.Axis(labels=False)),
    # we add a tooltip with the country's name
    tooltip = "country"    
).properties( width=150, height=150).repeat(
    # specify which data columns are used
    column=cols,
    row=cols
)

alt.RepeatChart(...)

✏️ *Create a scatterplot matrix of another dataset! Note that there are probably too many dimensions in the `oecd` to be displayed at once.*

### Correlation heatmap

To examine how the dimensions correlate across a larger dataset, we can visualize the pairwise correlation values in a compact matrix display that we might call a correlation heatmap or heat table. For this we will turn the correlation table we generated above into a DataFrame to be visualized:

In [28]:
corr = oecd.corr()

# first we reset the index and call it dim1
corr = corr.reset_index().rename(columns={'index': 'dim1'})

# turn correlation data into long form 
corr = pd.melt(corr, id_vars="dim1", var_name='dim2', value_name='corr')

# add a label column for rounded correlation values
corr['label'] = corr['corr'].map('{:.1f}'.format)

Now we have the pairwise correlation data in shape that we can visualize:

In [29]:
corr

,dim1,dim2,corr,label
0,dwellings without basic facilities,dwellings without basic facilities,1.000000,1.0
1,housing expenditure,dwellings without basic facilities,-0.489839,-0.5
2,rooms per person,dwellings without basic facilities,-0.567042,-0.6
3,household net adjusted disposable income,dwellings without basic facilities,-0.593321,-0.6
4,household net financial wealth,dwellings without basic facilities,-0.388205,-0.4
...,...,...,...,...
571,life satisfaction,time devoted to leisure and personal care,0.169276,0.2
572,assault rate,time devoted to leisure and personal care,-0.065735,-0.1
573,homicide rate,time devoted to leisure and personal care,-0.249768,-0.2
574,employees working very long hours,time devoted to leisure and personal care,-0.666652,-0.7


In [30]:
# we create a layered chart, with the base taking in the correlation data corr
# and the basic layout based on the dimensions
base = alt.Chart(corr).encode(
    x='dim1:O',
    y='dim2:O'    
).properties(width=500, height=500)

# a textual layer displaying rounded correlation values
text = base.mark_text().encode( text='label' )

# heatmap of the correlation values
plot = base.mark_rect().encode(
    color='corr:Q'
)

# both layers are combined
plot + text

alt.LayerChart(...)

### Multiple views

While we can accommodate multiple data dimensions in the various visual channels, it is also possible to separate out the data dimensions into multiple coordinated visualizations. Here we are now simply looking at the three data dimensions retrieved from Wikidata: population, area size, and GDP.

In [31]:
# to coordinate hover highlights we create a selection
selection = alt.selection_point(on='mouseover', fields=['country'])

# the definitions of the base are used by the three sub-charts
base = alt.Chart(wikidata).mark_bar().encode(
  # adjust opacity based on hover selection
  opacity=alt.condition(selection, alt.value(1), alt.value(.5)),
  x = alt.X("country:O").sort("-y").axis(None),
  tooltip=['country','population', 'area', 'gdp']
).properties(
    width=600, height=150
).add_params(selection)

# create a chart for each dimension
pop = base.encode(y = "population")
area = base.encode(y = "area")
gdp = base.encode(y = "gdp")

# combine them with ampersands
pop & area & gdp

alt.VConcatChart(...)

✏️ *Create a multi-view visualization with the `multiple` dataset:*

## Sources

Tutorials
- [Combining DataFrames with Pandas](https://datacarpentry.org/python-ecology-lesson/05-merging-data/)
- [Where do Mayors Come From: Querying Wikidata with Python and SPARQL](https://janakiev.com/blog/wikidata-mayors/)


Documentation
- [Merge, join, and concatenate — pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)
- [UMAP: Uniform Manifold Approximation and Projection for Dimension Reduction](https://umap-learn.readthedocs.io)
- [Altair Repeated Charts](https://altair-viz.github.io/user_guide/compound_charts.html#repeated-charts)




